In [2]:
import numpy as np
import pandas as pd

### dataset, cur_fold, r, lr, num_iter= sys.argv[1:]


In [3]:
err = {}
for dataset in [1, 2]:
    err[dataset] = {}
    for cur_fold in range(5):
        err[dataset][cur_fold] = {}
        for r in range(1 ,21):
            err[dataset][cur_fold][r] = {}
            for lr in [0.01, 0.1, 1.0, 2.0]:
                err[dataset][cur_fold][r][lr] = {}
                for num_iter in range(100, 2600, 400):
                    err[dataset][cur_fold][r][lr][num_iter] = \
                    np.load("../../baseline/mtf-nested/mtf-error-{}-{}-{}-{}-{}.npy".format(dataset, cur_fold, r, lr, num_iter)).item()

FileNotFoundError: [Errno 2] No such file or directory: '../../baseline/mtf-nested/mtf-error-1-0-1-0.01-100.npy'

In [4]:
iter = pd.DataFrame(err[1][0][3][0.01]).mean().idxmin()

In [5]:
pd.DataFrame(err[1][0][3][0.01]).mean()[2500]

177.28915755534743

In [6]:
best_r = {}
best_lr = {}
best_iter = {}
for dataset in [1, 2]:
    best_r[dataset] = {}
    best_lr[dataset] = {}
    best_iter[dataset] = {}
    for cur_fold in range(5):
        min_error = np.inf
        for r in range(1, 21):
            for lr in [0.01, 0.1, 1.0, 2.0]:
                iter = pd.DataFrame(err[dataset][cur_fold][r][lr]).mean().idxmin()
                error = pd.DataFrame(err[dataset][cur_fold][r][lr]).mean()[iter]
                if error < min_error:
                    min_error = error
                    best_r[dataset][cur_fold] = r
                    best_lr[dataset][cur_fold] = lr
                    best_iter[dataset][cur_fold] = iter

In [7]:
pd.DataFrame(best_r)

,1,2
0,20,16
1,18,13
2,17,13
3,10,3
4,20,15


In [8]:
pd.DataFrame(best_lr)

,1,2
0,2.0,1.0
1,1.0,2.0
2,2.0,0.1
3,2.0,2.0
4,1.0,1.0


In [9]:
pd.DataFrame(best_iter)

,1,2
0,100,1700
1,1700,500
2,100,2500
3,2100,100
4,2500,2100


In [17]:
import sys
sys.path.append("../code/")
from dataloader import APPLIANCE_ORDER, get_train_test
from tensor_custom_core import stf_4dim, stf_4dim_time


In [18]:
def stf(dataset, cur_fold, r=2, lr=1, num_iter=100):
    num_folds=5
    train, test = get_train_test(dataset, num_folds=num_folds, fold_num=cur_fold)
    test_copy = test.copy()
    test_copy[:, 1:, :, :] = np.NaN
    train_test = np.concatenate([train, test_copy])
    H, A, D, T = stf_4dim(tensor=train_test, r=r, lr=lr, num_iter=num_iter)
    pred = np.einsum("Hr, Ar, Dr, Tr ->HADT", H, A, D, T)[len(train):, 1:, :, :]
    
    return pred

### dataset1

In [20]:
dataset = 1
out = []
num_fold=5
for cur_fold in range(5):
    b_r= best_r[dataset][cur_fold]
    b_lr = best_lr[dataset][cur_fold]
    b_iter = best_iter[dataset][cur_fold]
    
    pred = stf(dataset, cur_fold, b_r, b_lr, b_iter)
    
    out.append(pred)

798.933843056 0
378.803348099 50
774.407629853 0
365.00344319 50
348.621090837 100
342.167086754 150
338.306065278 200
336.303861986 250
335.103598744 300
334.344257071 350
333.821063507 400
333.41211898 450
333.067870084 500
332.768277648 550
332.50458815 600
332.263785697 650
332.035411527 700
331.81530579 750
331.606187673 800
331.413104578 850
334.61270702 900
331.063309943 950
330.897317442 1000
330.721851726 1050
330.531779888 1100
330.33091099 1150
330.126299526 1200
329.928018319 1250
329.750957264 1300
329.597514526 1350
329.465269089 1400
329.350633183 1450
329.251723463 1500
329.163389804 1550
329.084942314 1600
329.014487962 1650
758.386331002 0
370.68698227 50
711.568421186 0
367.737972621 50
348.901025068 100
344.276390236 150
342.733977494 200
341.997188568 250
341.494687181 300
341.038885002 350
340.578743822 400
340.144819979 450
339.782136944 500
339.511915195 550
339.310314185 600
339.146562617 650
339.01095118 700
338.914544877 750
338.840747355 800
343.917095059 85

In [24]:
cont_out = np.concatenate(out)
tensor = np.load('../2015-5appliances.numpy.npy')
gt = tensor[:, 1:, :, :]

In [26]:
from sklearn.metrics import mean_absolute_error
err_stf = {APPLIANCE_ORDER[i+1]:mean_absolute_error(cont_out[:, i,:,:].flatten(), 
                                                                       gt[:, i, :, :].flatten()) for i in range(pred.shape[1])}

In [27]:
err_stf

{'dr': 117.55267578113788,
 'dw': 27.461522878707523,
 'fridge': 43.819964033418202,
 'hvac': 331.63421940163073,
 'mw': 10.488311233484415}

### dataset 2

In [ ]:
dataset = 2
out = []
num_fold=5
for cur_fold in range(5):
    b_r= best_r[dataset][cur_fold]
    b_lr = best_lr[dataset][cur_fold]
    b_iter = best_iter[dataset][cur_fold]
    
    pred = stf(dataset, cur_fold, b_r, b_lr, b_iter)
    
    out.append(pred)

1252.33308992 0
527.959217355 50
497.913017577 100
485.598775016 150
480.795097588 200
478.138420724 250
476.488134263 300
475.32242377 350
474.349240402 400
473.447005509 450
472.608111016 500
471.894729038 550
471.32460406 600
470.90972481 650
470.597483214 700
470.359519399 750
470.173643753 800
470.025461654 850
469.903689551 900
470.2406338 950
469.717039465 1000
469.631922545 1050
469.559896712 1100
469.493289488 1150
469.431321994 1200
469.373852816 1250
469.320427133 1300
469.270083076 1350
469.222136247 1400
469.176311398 1450
469.132448512 1500
469.090392435 1550
469.051494132 1600
469.01667089 1650
1130.47780132 0
522.329721924 50
492.205349408 100
481.823028392 150
477.514654514 200
475.683895015 250
476.886950426 300
472.657812498 350
471.911310251 400
471.344719487 450
1277.35094154 0
1027.75940997 50
745.517658982 100
626.701261412 150
587.095526992 200
572.455908563 250
565.62380263 300
561.280690543 350
557.730145748 400
554.413956037 450
551.20782432 500
548.044038134